# Lab 9

In this lab, you will explore spatial data analysis using Python and DuckDB. You'll work with real-world datasets, ranging from global country statistics to specific building datasets. This will give you a practical understanding of handling, analyzing, and visualizing spatial data.

**Submission requirements**

1. **HTML Version:** Submit an HTML version of your notebook. Ensure all code outputs are visible. (Export via VS Code: Notebook > Export > HTML).
2. **Colab Link:** Provide a link to your notebook hosted on Google Colab for interactive review.

## Setup

Ensure you have DuckDB and Leafmap installed. Run the following command if needed:

In [ ]:
# %pip install duckdb leafmap

In [ ]:
import duckdb
import leafmap

## Question 1

Connect to a duckdb database and install the `httpfs` and `spatial` extensions

In [ ]:
con = duckdb.connect()  # defaults is memory
con.install_extension("httpfs")
con.load_extension("httpfs")

In [ ]:
con.install_extension("spatial")
con.load_extension("spatial")

## Question 2

Download the [Admin 0 – Countries](https://www.naturalearthdata.com/downloads/10m-cultural-vectors/) vector dataset from Natural Earth using the `leafmap.download_file()` function.

In [ ]:
url = "https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/cultural/ne_10m_admin_0_countries.zip"
# leafmap.download_file(url)

## Question 3

Create a new table in your database called `countries` and load the data from the downloaded country shapefile into it.

In [ ]:
con.sql(
    """
        CREATE TABLE IF NOT EXISTS countries AS 
        SELECT * FROM ST_Read('ne_10m_admin_0_countries.shp')
        """
)

In [ ]:
con.table("countries")

Calculate the total population of all countries in the database using the `POP_EST` column.

In [ ]:
con.sql("SELECT SUM(POP_EST) AS total_pop FROM countries;")

Show the top 10 countries with the largest population.

In [ ]:
con.sql(
    """
        SELECT SOVEREIGNT, POP_EST FROM countries
        ORDER BY POP_EST DESC
        LIMIT 10;
        """
)

Select countries in Europe with a population greater than 10 million and order them by population in descending order.

In [ ]:
con.sql(
    """
        SELECT * FROM countries
        WHERE POP_EST > 10000000 AND REGION_UN = 'Europe'
        ORDER BY POP_EST DESC;
        """
)

Save the results of the previous query as a new table called `europe`.

In [ ]:
con.sql(
    """
        CREATE TABLE IF NOT EXISTS europe AS 
        SELECT * FROM countries
        WHERE POP_EST > 10000000 AND REGION_UN = 'Europe'
        ORDER BY POP_EST DESC;
        """
)

In [ ]:
con.table("europe")

Export the `europe` table as a GeoJSON file.

In [ ]:
con.sql("COPY europe TO 'europe.geojson' WITH (FORMAT GDAL, DRIVER 'GeoJSON')")

## Question 4

Create a table called `text_zones` and load the data from the [taxi_zones.parquet](https://beta.source.coop/cholmes/nyc-taxi-zones/taxi_zones.parquet) into it.

In [ ]:
url = "https://data.source.coop/cholmes/nyc-taxi-zones/taxi_zones.parquet"

In [ ]:
con.sql(f"FROM '{url}'")

In [ ]:
con.sql(
    f"""
CREATE or REPLACE TABLE text_zones AS
SELECT * EXCLUDE geometry, ST_GeomFromWKB(geometry) AS geometry FROM
        '{url}'
"""
)

In [ ]:
con.table("text_zones")

Find out the unique values in the `borough` column and order them alphabetically.

In [ ]:
con.sql("SELECT DISTINCT borough FROM text_zones ORDER BY borough;")

Export the `text_zones` table as a parquet file.

In [ ]:
con.sql("COPY text_zones TO 'text_zones.parquet' (FORMAT PARQUET)")

## Question 5

Explore the [Google Open Buildings](https://beta.source.coop/cholmes/google-open-buildings/v2/geoparquet-admin1/) and select a country of your choice with relatively small number of buildings (i.e., small file size). Get the three character country code and replace `[COUNTRY_NAME]` in the following path with the country code. Use it to load all the parquet files for the selected country into a new table called `buildings`.

`s3://us-west-2.opendata.source.coop/google-research-open-buildings/v2/geoparquet-admin1/country=[COUNTRY_NAME]/*.parquet`

In [ ]:
url = "https://data.source.coop/cholmes/google-open-buildings/v2/geoparquet-admin1/country=DJI/Obock.parquet"

In [ ]:
con.sql(
    f"""
CREATE or REPLACE TABLE buildings AS
SELECT * EXCLUDE geometry, ST_GeomFromWKB(geometry) AS geometry FROM
        '{url}'
"""
)

In [ ]:
con.sql("FROM buildings")

Find out the number of buildings in the selected country.

In [ ]:
con.sql("SELECT COUNT(area_in_meters) AS total_buildings FROM buildings;")

Find out the total area of all buildings in the selected country.

In [ ]:
con.sql("SELECT SUM(area_in_meters) AS total_area FROM buildings;")

Export the `buildings` table as a GeoPackage file.

In [ ]:
con.sql("COPY buildings TO 'buildings.gpkg' WITH (FORMAT GDAL, DRIVER 'GPKG')")